In [83]:
import pandas as pd
import json
from datetime import datetime

In [84]:
class my_dictionary(dict): 
  
    # __init__ function 
    def __init__(self): 
        self = dict() 
          
    # Function to add key:value 
    def add(self, key, value): 
        self[key] = value 


# cleaning data

In [85]:

import re
df = pd.read_csv('goodreads_output3.csv')
dfwin = pd.read_csv('AwardDates.csv')
data = dict()
with open('mona/newDates.json') as json_file:
    data = json.load(json_file)
for d in data: 
    date = d['date']
    result = re.search('Published(.*)', date)
    partial = result.group(1)
    if('by' in partial) :
        partial = re.search('(.*)by', partial)
        d['date'] = partial.group(1).lstrip().rstrip()
    else:
        d['date'] = partial.lstrip().rstrip()

with open('mona/new.json', 'w') as new:
    json.dump(data, new)

In [86]:
# calculating dates and matching isbn
data = dict()
with open('mona/new.json') as json_file:
    data = json.load(json_file)
hi = my_dictionary()
h2 = my_dictionary()
h3 = list()
for d in data:

    ds = str(d['date']).split()
    if len(ds)>1:
        ds[1]=ds[1].replace('th','').replace('st','').replace('rd','').replace('nd','')
        d['date'] = ''.join(str(e)+' ' for e in ds)
    if len(ds)>3:
         d['date'] = ''.join([ds[0],' ',ds[1],' ',ds[2],' '])
    try:
        d['date']=datetime.strptime(d['date'], '%B %d %Y ')
    except ValueError :
        try:
           d['date']= datetime.strptime(d['date'], '%B %Y ')
        except ValueError :
           d['date']= datetime.strptime(d['date'], '%Y')
    if(df[df['url']==d['url']].empty):
        continue
    year=df[df['url']==d['url']]['year'].iloc[0]
    win=datetime.strptime(dfwin[dfwin['year'] ==year]['Winners Announced'].iloc[0],'%m/%d/%Y')
    if((win -  d['date']).days<1):
        d['date']=(win -  d['date']).days + 365
    else:
        d['date']=(win -  d['date']).days 
#     if (d['name'] == 'Calling Dr. Laura'):
#         print(d['date'])
#         print(win)
    hi.add(d['url'], d['date'])
    h2.add(d['url'],d['isbn'])
    h3.append(d['isbn'])
    


# merging data and adding 'times_appred_NYTime','avgrank_NYTime','bestRank','listnames' to are coulems

In [87]:
dfnew=pd.DataFrame(hi.items(), columns=['url', 'daystocom'])
dfnew1 = pd.DataFrame(h2.items(), columns=['url', 'isbn'])
df.merge(dfnew,on='url', how='left').merge(dfnew1,on='url', how='left').to_csv("withdays.csv")
df = pd.read_csv('withdays.csv')
df = df.reindex(df.columns.tolist() + ['times_appred_NYTime','avgrank_NYTime','bestRank','listnames'], axis=1)


In [ ]:
df

In [ ]:
def 

reading newYork Data and adding to feilds

In [ ]:
data = dict()
data1 = dict()
with open('newYork Data/res1.json') as json_file:
    data = json.load(json_file)
print(len(data))
with open('newYork Data/res2.json') as json_file:
    data1 = json.load(json_file)
data.update(data1)
print(len(data))
with open('newYork Data/res4.json') as json_file:
    data1 = json.load(json_file)
data.update(data1)
print(len(data))
with open('newYork Data/res5.json') as json_file:
    data1 = json.load(json_file)
data.update(data1)
print(len(data))
with open('newYork Data/res6.json') as json_file:
    data1 = json.load(json_file)
data.update(data1)
print(len(data))
with open('newYork Data/res7.json') as json_file:
    data1 = json.load(json_file)
data.update(data1)
print(len(data))
with open('newYork Data/res8.json') as json_file:
    data1 = json.load(json_file)
data.update(data1)
print(len(data))
with open('newYork Data/res9.json') as json_file:
    data1 = json.load(json_file)
data.update(data1)
print(len(data))
with open('newYork Data/res10.json') as json_file:
    data1 = json.load(json_file)
data.update(data1)
print(len(data))
for i in h3:
    try:
        if i =='English': continue
        if len(data[i]['results'])==0:
                df.loc[df['isbn']==i,['times_appred_NYTime']] = 0
                df.loc[df['isbn']==i,['avgrank_NYTime']] =float('inf')
                df.loc[df['isbn']==i,['bestRank']] = float('inf')
                df.loc[df['isbn']==i,['listnames']] = str(0)
                continue
        for j in data[i]['results']:
            bestrank=float('inf')
            num=0
            count = 0 
            listname = set()
            if len(j['ranks_history'])==0:
                df.loc[df['isbn']==i,['times_appred_NYTime']] =0
                df.loc[df['isbn']==i,['avgrank_NYTime']] =float('inf')
                df.loc[df['isbn']==i,['bestRank']] = float('inf')
                df.loc[df['isbn']==i,['listnames']] = str(0)
                continue
            year = df[df['isbn']==i]['year']
#             print(year)
            for k in year:
                win=datetime.strptime(dfwin[dfwin['year'] ==k]['Winners Announced'].iloc[0],'%m/%d/%Y')
                for z in j['ranks_history']:
                    if(datetime.strptime(z['published_date'], '%Y-%m-%d')<win):
                        num+= z['rank']
                        listname.add(z['list_name'])
                        if bestrank>z['rank']:
                            bestrank =z['rank']
                        count  = count + 1
                    if count>0:
                        num=num/count
                        df.loc[df['isbn']==i,['times_appred_NYTime']] = count
                        df.loc[df['isbn']==i,['avgrank_NYTime']] =num
                        df.loc[df['isbn']==i,['bestRank']] = bestrank
                        df.loc[df['isbn']==i,['listnames']] = str(len(listname))
                    else:
                        df.loc[df['isbn']==i,['times_appred_NYTime']] = 0
                        df.loc[df['isbn']==i,['avgrank_NYTime']] =float('inf')
                        df.loc[df['isbn']==i,['bestRank']] = float('inf')
                        df.loc[df['isbn']==i,['listnames']] = str(0)
                        
    except KeyError :
        print(i)
        continue


438
870
1295
1405
1810
2245
2690
3146
3350


saving data

In [ ]:
df.to_csv("withdays.csv")

cheak nall values

In [ ]:
df = pd.read_csv('withdays.csv')

def nans(df): return df[df.isnull().any(axis=1)]
nans(df)

read data from json 11 with different struct and saving to csv

In [ ]:
with open('newYork Data/res11.json') as json_file:
    data = json.load(json_file)
for i in data:
    if len(data[i]['results'])==0:
        df.loc[df['url']==i,['times_appred_NYTime']] = float('inf')
        df.loc[df['url']==i,['avgrank_NYTime']] =float('inf')
        df.loc[df['url']==i,['bestRank']] = float('inf')
        df.loc[df['url']==i,['listnames']] = str(0)
        continue
    for j in data[i]['results']:
        bestrank=float('inf')
        num=0
        listname = set()
        if len(j['ranks_history'])==0:
                df.loc[df['url']==i,['times_appred_NYTime']] = float('inf')
                df.loc[df['url']==i,['avgrank_NYTime']] =float('inf')
                df.loc[df['url']==i,['bestRank']] = float('inf')
                df.loc[df['url']==i,['listnames']] = str(0)
                continue
        year = df[df['url']==i]['year']
        for k in year:
            win=datetime.strptime(dfwin[dfwin['year'] ==k]['Winners Announced'].iloc[0],'%m/%d/%Y')
            for z in j['ranks_history']:
                if(datetime.strptime(z['published_date'], '%Y-%m-%d')<win):
                    num+= z['rank']
                    listname.add(z['list_name'])
                    if bestrank>z['rank']:
                        bestrank =z['rank']
                    count  = count + 1
            num=num/count
            df.loc[(df['url']==i)&(df['year']==k),['times_appred_NYTime']] = count
            df.loc[(df['url']==i)&(df['year']==k),['avgrank_NYTime']] =num
            df.loc[(df['url']==i)&(df['year']==k),['bestRank']] = bestrank
            df.loc[(df['url']==i)&(df['year']==k),['listnames']] = str(len(listname))
df.to_csv("withdays.csv")


saving nulls in file

In [ ]:
def nans(df): return df[df.isnull().any(axis=1)]
nans(df).to_csv("nulls.csv")



In [ ]:
# dfnew=pd.DataFrame(hi.items(), columns=['url', 'daystocom'])
# dfnew1 = pd.DataFrame(h2.items(), columns=['url', 'isbn'])
# # dfnew2 = pd.DataFrame(h5.items(), columns=['isbn', 'times_appred_NYTime'])
# # dfnew3 = pd.DataFrame(h4.items(), columns=['isbn', 'avgrank_NYTime'])
# # dfnew4 = pd.DataFrame(h6.items(), columns=['isbn', 'bestRank'])
# # dfnew5 = pd.DataFrame(h7.items(), columns=['isbn', 'listnames'])
# df.merge(dfnew,on='url', how='left').merge(dfnew1,on='url', how='left').to_csv("withdays.csv")
# # .merge(dfnew3,on='isbn', how='left').merge(dfnew4,on='isbn', how='left').merge(dfnew5,on='isbn', how='left').to_csv("withdays.csv")